In [234]:
using LowLevelFEM

In [235]:
gmsh.initialize()

In [236]:
gmsh.open("cube1.geo")

Info    : Reading 'cube1.geo'...
Info    : Meshing 1D...
Info    : [  0%] Meshing curve 1 (Line)
Info    : [ 10%] Meshing curve 2 (Line)
Info    : [ 20%] Meshing curve 3 (Line)
Info    : [ 30%] Meshing curve 4 (Line)
Info    : [ 40%] Meshing curve 5 (Line)
Info    : [ 50%] Meshing curve 6 (Line)
Info    : [ 60%] Meshing curve 7 (Line)
Info    : [ 60%] Meshing curve 8 (Line)
Info    : [ 70%] Meshing curve 9 (Line)
Info    : [ 80%] Meshing curve 10 (Line)
Info    : [ 90%] Meshing curve 11 (Line)
Info    : [100%] Meshing curve 12 (Line)
Info    : Done meshing 1D (Wall 0.00370138s, CPU 0.002898s)
Info    : Meshing 2D...
Info    : [  0%] Meshing surface 1 (Transfinite)
Info    : [ 20%] Meshing surface 2 (Transfinite)
Info    : [ 40%] Meshing surface 3 (Transfinite)
Info    : [ 60%] Meshing surface 4 (Transfinite)
Info    : [ 70%] Meshing surface 5 (Transfinite)
Info    : [ 90%] Meshing surface 6 (Transfinite)
Info    : Done meshing 2D (Wall 0.00382445s, CPU 0.003823s)
Info    : Meshing 3D..

In [237]:
mat = material("cube")
prob = Problem([mat])

Problem("cube1", :Solid, 3, 3, LowLevelFEM.Material[LowLevelFEM.Material("cube", :Hooke, 200000.0, 0.3, 7.85e-9, 45.0, 4.2e8, 1.2e-5, 115384.61538461536, 76923.07692307692, 166666.66666666666, 1.0)], 1.0, 29791)

In [238]:
supp = displacementConstraint("rear", ux=0, uy=0, uz=1)
load1 = load("front", fz=100000)

("front", 0, 0, 100000)

In [239]:
f = loadVector(prob, [load1]);

In [240]:
using SparseArrays

In [241]:
function solveCondensed(prob, load, supp)
    K = stiffnessMatrix(prob)
    f = loadVector(prob, load)
    free = freeDoFs(prob, supp)
    fixed = constrainedDoFs(prob, supp)
    u = copy(f)
    fill!(u.a, 0.0)
    applyBoundaryConditions!(u, supp)
    f_kin = K.A[:, fixed] * u.a[fixed]
    u.a[free] = K.A[free, free] \ (f.a[free] - f_kin[free])
    return u
end

solveCondensed (generic function with 1 method)

In [242]:
function solveZeroRows(prob, load, supp)
    K = stiffnessMatrix(prob)
    f = loadVector(prob, load)
    fixed_dofs = constrainedDoFs(prob, supp)
    u2 = copy(f)
    fill!(u2.a, 0.0)
    applyBoundaryConditions!(u2, supp)
    f_kin = K.A[:, fixed_dofs] * u2.a[fixed_dofs]
    fixed = falses(size(K.A, 1))
    fixed[fixed_dofs] .= true
    for col in 1:size(K.A, 2)
        colrange = K.A.colptr[col]:(K.A.colptr[col+1]-1)
        for rowidx in colrange
            row = K.A.rowval[rowidx]
            if fixed[row]
                K.A.nzval[rowidx] = 0.0
            elseif fixed[col]
                K.A.nzval[rowidx] = 0.0
            end
        end
    end
    for dof in fixed_dofs
        K.A[dof, dof] = 1.0  # ha létezett korábban
        f.a[dof] = u2.a[dof]
        f_kin[dof] = 0.0
    end
    dropzeros!(K.A)
    u2.a .= K.A \ (f.a - f_kin)
    return u2
end

solveZeroRows (generic function with 1 method)

In [243]:
using BenchmarkTools

In [244]:
@btime solveCondensed(prob, [load1], [supp]);

  81.434 s (59035006 allocations: 13.45 GiB)


In [245]:
@btime solveDisplacement(prob, [load1], [supp]);

  83.952 s (14982935 allocations: 12.59 GiB)


In [246]:
@btime solveZeroRows(prob, [load1], [supp]);

  83.693 s (43630060 allocations: 13.12 GiB)


In [247]:
u = solveCondensed(prob, [load1], [supp]);

In [248]:
u2 = solveZeroRows(prob, [load1], [supp]);

In [249]:
u3 = solveDisplacement(prob, [load1], [supp]);

In [250]:
showDoFResults(f)
showDoFResults(u)

1

In [251]:
showDoFResults(u2)

2

In [252]:
showDoFResults(u3)

3

In [253]:
openPostProcessor()

-------------------------------------------------------
Version       : 4.13.1
License       : GNU General Public License
Build OS      : Linux64-sdk
Build date    : 19700101
Build host    : amdci7.julia.csail.mit.edu
Build options : 64Bit ALGLIB[contrib] ANN[contrib] Bamg Blossom Cairo DIntegration Dlopen DomHex Eigen[contrib] Fltk GMP Gmm[contrib] Hxt Jpeg Kbipack LinuxJoystick MathEx[contrib] Mesh Metis[contrib] Mmg Mpeg Netgen Nii2mesh ONELAB ONELABMetamodel OpenCASCADE OpenCASCADE-CAF OpenGL OpenMP OptHom Parser Plugins Png Post QuadMeshingTools QuadTri Solver TetGen/BR TinyXML2[contrib] Untangle Voro++[contrib] WinslowUntangler Zlib
FLTK version  : 1.3.8
OCC version   : 7.7.2
Packaged by   : root
Web site      : https://gmsh.info
Issue tracker : https://gitlab.onelab.info/gmsh/gmsh/issues
-------------------------------------------------------


XRequest.18: BadValue 0x0


In [254]:
gmsh.finalize()